# 1 - VISTAS

Las vistas en `MySQL` son tablas virtuales que se crean a partir de una consulta más o menos complicada.

Las vistas se pueden usar igual que una tabla cualquiera de nuestra base de datos: puedo usarla en un join, en una subconsulta, en un `SELECT`, etc...

Pero la principal diferencia es que las vistas no almacenan datos, sino que son una consulta que se ejecuta cada vez que se usa la vista.(la consulto, la junto, la filtro...)

Por lo que si modifico los datos de la tabla o tablas que intervienen en la vista, la vista se actualizará automáticamente.


Para ***crear una vista***, usamos la siguiente sintaxis:
```sql
CREATE VIEW nombre_vista AS
    SELECT columnas
    FROM tablas
    WHERE condiciones;
```
Donde:

- `CREATE VIEW` es la sentencia para crear una vista.
- `nombre_vista` es el nombre que le queremos dar a la vista.
- `AS` es la palabra clave que indica que vamos a definir la vista.


Vamos a crear una vista que nos indique sobre la base da datos de `ClassicModels` para el último mes de cada una de las oficinas, cuantos pedidos se han realizado, importe total de los pedidos y clientes distintos que han realizado pedidos.

|Año|Mes|Ciudad|ClientesUnicos|Pedidos|TotalVentas(€)|
|---|---|------|--------------|-------|-----------|
|   |   |      |              |       |           |
|   |   |      |              |       |           |
|   |   |      |              |       |           |
|   |   |      |              |       |           |
|   |   |      |              |       |           |
|   |   |      |              |       |           |

Para ello lo primero es crear la consulta que nos devuelva lo que queremos ver en la vista:

1 - Vamos a ver que tablas necesitamos para hacer la consulta

2 - Veamos por donde agrupar lo datos

3 - Veamos que funciones de agregación/ventana necesitamos

4 - Veamos que campos necesitamos


```sql
CREATE VIEW estado_tiendas AS
SELECT
	año, mes, city, clientes_distintos, pedidos, total_ventas
FROM
(SELECT
	officeCode
    , YEAR(orderdate) año
    , MONTH(orderDate) mes
    , ofi.city city
    , COUNT(DISTINCT (customerNumber)) AS clientes_distintos
    , COUNT(o.orderNumber) AS pedidos
    , SUM(quantityOrdered * priceEach) AS total_ventas
    , row_number() OVER (PARTITION BY officeCode ORDER BY YEAR(orderdate) DESC, MONTH(orderDate) DESC)
    as num_linea
FROM orders o
JOIN orderdetails od USING (orderNumber)
JOIN customers c USING (customerNumber)
JOIN employees e
	ON e.employeeNumber = c.salesRepEmployeeNumber
JOIN offices ofi USING(officeCode)
GROUP BY 1,2,3
-- ORDER BY 1,2,3
) a
WHERE num_linea = 1
ORDER BY total_ventas
```

# 2 - TABLAS TEMPORALES

Las tablas temporales son tablas que se crean en memoria y que se eliminan cuando se cierra la conexión a la base de datos.

Las tablas temporales se pueden usar igual que una tabla cualquiera de nuestra base de datos: puedo usarla en un join, en una subconsulta, en un `SELECT`, etc...pero solo están disponibles para la conexión que las ha creado, mientras ésta esté abierta.

Las tablas temporales, además, pueden ser de dos tipos:

- ***Tablas temporales locales***: Son tablas que solo están disponibles para la conexión que las ha creado, mientras ésta esté abierta.
- ***Tablas temporales globales***: Son tablas que están disponibles para todas las conexiones que se hagan a la base de datos, mientras la conexión que las ha creado esté abierta.

Para ***crear una tabla temporal***, usamos la siguiente sintaxis:

```sql
CREATE TEMPORARY TABLE nombre_tabla_temporal
(
    nombre_columna tipo_dato,
    nombre_columna2 tipo_dato,
    ...
);
```

Vamos a crear una tabla temporal que contenga 3 columnas una con el año , otra con la ciudad y otra total_ventas:

![image.png](attachment:image.png)

Ya hemos creado nuestra tabla temporal en nuestra database, ahora vamos a insertar los datos que queremos que contenga:

```sql
INSERT INTO nombre_tabla_temporal (nombre_columna, nombre_columna, ...)
VALUES (valor, valor, ...),
       (valor, valor, ...),
       ...
```

Igual que lo haríamos con una tabla normal. Pero este método es muy tedioso básicamente tendríamos que picar a mano y separado por comas y por filas todos los datos que quisieramos insertar...trabajo de chinos.

Por suerte, podemos hacerlo de otra manera, usando una consulta:

```sql
INSERT INTO nombre_tabla_temporal (nombre_columna1, nombre_columna2, ...)
SELECT nombre_columna1, nombre_columna2, ...
FROM nombre_tabla;
```

Vamos a ver como podemos crear una tabla desde un `SELECT`:

```sql
CREATE Table tabla_molona as (
SELECT 	*
FROM orders
JOIN orderdetails USING (orderNumber)
JOIN customers USING (customerNumber)
)
```

De esta manera, creamos una tabla con el nombre que le indiquemos en `tabla_molona` con las columnas que le indiquemos en la instrucción SELECT y con los datos que le indiquemos en que serán los que nos devuelva la consulta que pongamos en las tablas de nuestro FROM

Nos evitamos tener que declarar las tablas, sus columnas y los tipos de datos de cada columna.


# START TRANSACTION:

La sentencia `START TRANSACTION` es la que inicia una transacción en `MySQL`.

Una transacción es un conjunto de operaciones que se ejecutan como si fueran una sola operación. Es decir, si una de las operaciones falla, todas las operaciones se deshacen.

Por ejemplo, si tengo una transacción que consiste en insertar un registro en una tabla y actualizar otro registro en otra tabla, si la actualización falla, el registro que he insertado en la primera tabla se deshace.

Lo importante de las transacciones, es que nos aseguramos que todas las operaciones se ejecutan o ninguna.

Además al meter nuestras modificaciones en una transacción, nos permite poder hacer un `ROLLBACK` si algo falla y no queremos que se apliquen los cambios. En caso de que todo vaya bien, podemos hacer un `COMMIT` para que se apliquen los cambios de manera definitiva.

Siempre iniciar la transacción cuando vamos a hacer una gran alteración de la base de datos como INSERT, ALTER, DELETE, DROP (que no tendréis permiso para ello), UPDATE...

Vamos a intentar borrar registros con los siguientes condiciones:
```sql
START  TRANSACTION;
DELETE orders
FROM orders
INNER JOIN orderdetails on orders.orderNumber= orderdetails.orderNumber
INNER JOIN customers USING (customerNumber)
INNER JOIN employees ON employees.employeeNumber = customers.salesRepEmployeeNumber
INNER JOIN offices USING(officeCode)    
WHERE officeCode = 7
AND YEAR(orderDate) = 2005;
SELECT *
FROM Ultimo_mes_tiendas;
```

Estamos borrando de la tabla `orders` todos los registros que cumplan las condiciones que le hemos indicado.
Es decir que pertenezcan a la oficina 7 y que sean del año 2005.

Si ejecutamos la consulta recibimos el siguiente mensaje:
```sql
Error Code: 1451. Cannot delete or update a parent row: a foreign key constraint fails (`classicmodels`.`orderdetails`, CONSTRAINT `orderdetails_ibfk_1` FOREIGN KEY (`orderNumber`) REFERENCES `orders` (`orderNumber`))
```

Esto es debido a que la tabla `orderdetails` tiene una clave foránea que hace referencia a la tabla `orders` y no podemos borrar registros de la tabla `orders` si hay registros en la tabla `orderdetails` que hagan referencia a ellos. Por lo que primero deberíamos borrar los registros de la tabla `orderdetails` que hagan referencia a los registros que queremos borrar de la tabla `orders`.

Si cambiáramos el orden de las tablas en el `DELETE` para que primero borrara de `orderdetails` y luego de `orders`, la consulta funcionaría.

```sql
START  TRANSACTION;
DELETE orderdetails
FROM orders
INNER JOIN orderdetails on orders.orderNumber= orderdetails.orderNumber
INNER JOIN customers USING (customerNumber)
INNER JOIN employees ON employees.employeeNumber = customers.salesRepEmployeeNumber
INNER JOIN offices USING(officeCode)    
WHERE officeCode = 1
AND YEAR(orderDate) = 2005;
SELECT *
FROM Ultimo_mes_tiendas;
```


Pero si nos fijamos, la tabla `Ultimo_mes_tiendas` es una vista que hemos creado anteriormente y que está basada en la tabla `orders`. Por lo que si borramos registros de `orders`, la vista `Ultimo_mes_tiendas` se actualizará automáticamente.

### RETO
Dentro de nuestra DB tenemos `payments` y `orders`.

Quiero obtener un línea que me arroje solo dos filas, una con el cliente que más nos debe y otra con el cliente al que más dinero le debemos.


|NombreEmpresa|IdCliente|ImportePagado|ImportePedidos|
|-------------|---------|-------------|--------------|
|             |         |             |              |
|             |         |             |              |